In [2]:
!ls autogpt

agent		    configurator.py  logs.py	  processing	spinner.py
app.py		    __init__.py      __main__.py  prompts	url_utils
auto_gpt_workspace  js		     main.py	  __pycache__	utils.py
cli.py		    json_utils	     memory	  setup.py	workspace
commands	    llm		     models	  singleton.py
config		    log_cycle	     plugins.py   speech


In [4]:
import json
import re


workflows = []
block = {"PLAN": []}
with open('logs/activity.log','r') as lines:
    for line in lines:
        if "INFO PANGUGPT THOUGHTS:" in line:
            block["PANGUGPT THOUGHTS"] = re.search(r'INFO PANGUGPT THOUGHTS: (.*)', line).group(1)
        elif "INFO REASONING:" in line:
            block["REASONING"] = re.search(r'INFO REASONING: (.*)', line).group(1)
        elif "INFO -" in line:
            block["PLAN"].append(re.search(r'INFO -  (.*)', line).group(1))
        elif "INFO SPEAK:" in line:
            block["SPEAK"] = re.search(r'INFO SPEAK: (.*)', line).group(1)
        elif "DEBUG" in line:
            block["ACTION"] = json.loads(re.search(r'DEBUG  (.*)', line).group(1))
            output.append(block)
            block = {"PLAN": []}

AttributeError: 'NoneType' object has no attribute 'group'

In [107]:
split_by_action_lines = []
one_action_resource = {'ACTION':[],'RESULT':None}
with open('logs/activity.log','r') as lines:
    for line in lines:
        if "NEXT ACTION:" not in line:
            one_action_resource['ACTION'].append(line)
        else:
            one_action_resource['RESULT']=re.search(r'NEXT ACTION: (.*)', line).group(1)
            one_action_resource['ACTION']=parse_one_action(one_action_resource['ACTION'])
            split_by_action_lines.append(one_action_resource)
            one_action_resource = {'ACTION':[],'RESULT':None}
            

In [113]:
_=[print(t['RESULT']) for t in split_by_action_lines]

 COMMAND = google  ARGUMENTS = {'query': 'Pangu Large Model'}
 COMMAND = browse_website  ARGUMENTS = {'url': 'https://www.huawei.com/en/press-events/news/2021/7/huawei-releases-pangu-large-model', 'question': 'What is the Pangu Large Model?'}
 COMMAND = google  ARGUMENTS = {'query': 'Pangu Large Model function and author'}
 COMMAND = google  ARGUMENTS = {'query': 'Pangu Large Model function and author'}
 COMMAND = read_file  ARGUMENTS = {'filename': '/home/zhangtianning/Documents/projects/Auto-GPT/autogpt/auto_gpt_workspace/pangu_large_model.txt'}
 COMMAND = write_to_file  ARGUMENTS = {'filename': '/home/zhangtianning/Documents/projects/Auto-GPT/autogpt/auto_gpt_workspace/pangu_large_model.txt', 'text': ''}
 COMMAND = google  ARGUMENTS = {'query': 'Pangu Large Model function and author'}
 COMMAND = google  ARGUMENTS = {'query': 'Pangu Large Model capabilities and applications'}
 COMMAND = read_file  ARGUMENTS = {'filename': '/home/zhangtianning/Documents/projects/Auto-GPT/autogpt/auto_

In [104]:
def parse_one_action(lines):
    output = []
    block = {}
    debug_flag = plan_flag = False
    debug_lines = []

    for line in lines:
        if 'INFO NEWS' in line:continue
        if "INFO PANGUGPT THOUGHTS:" in line:
            block["PANGUGPT THOUGHTS"] = re.search(r'INFO PANGUGPT THOUGHTS: (.*)', line).group(1)
        elif "INFO REASONING:" in line:
            block["REASONING"] = re.search(r'INFO REASONING: (.*)', line).group(1)
        elif "INFO PLAN:" in line:
            plan_flag = True
            block["PLAN"] = []
        elif plan_flag and "INFO -" in line:
            block["PLAN"].append(re.search(r'INFO -  (.*)', line).group(1))
        elif "INFO SPEAK:" in line:
            block["SPEAK"] = re.search(r'INFO SPEAK: (.*)', line).group(1)
            plan_flag = False
        elif "DEBUG" in line:
            debug_lines=[]
            debug_flag = True
            string = re.sub(r'[\s]* DEBUG [\s]*', '', line)
            debug_lines.append(re.search(r'DEBUG(.*)', line).group(1))
        elif debug_flag:
            if not re.match(r'^\d{4}-\d{2}-\d{2}', line):
                debug_lines.append(line)
            else:
                json_str = "".join(debug_lines)
                try:
                    json_pool = json.loads(json_str)
                except:
                    print(json_str)
                    json_pool = json_str
                block["DEBUG"] = json_pool  
                output.append(block)
                block = {}
                debug_flag = False
                debug_lines = []

    if debug_flag:
        block["DEBUG"] = json.loads("".join(debug_lines))
        output.append(block)
    return output

In [95]:
output = []
block = {}
debug_flag = plan_flag = False
debug_lines = []
with open('logs/activity.log','r') as lines:
    for line in lines:
        if 'INFO NEWS' in line:continue
        if "INFO PANGUGPT THOUGHTS:" in line:
            block["PANGUGPT THOUGHTS"] = re.search(r'INFO PANGUGPT THOUGHTS: (.*)', line).group(1)
        elif "INFO REASONING:" in line:
            block["REASONING"] = re.search(r'INFO REASONING: (.*)', line).group(1)
        elif "INFO PLAN:" in line:
            plan_flag = True
            block["PLAN"] = []
        elif plan_flag and "INFO -" in line:
            block["PLAN"].append(re.search(r'INFO -  (.*)', line).group(1))
        elif "INFO SPEAK:" in line:
            block["SPEAK"] = re.search(r'INFO SPEAK: (.*)', line).group(1)
            plan_flag = False
        elif "DEBUG" in line:
            debug_lines=[]
            debug_flag = True
            string = re.sub(r'[\s]* DEBUG [\s]*', '', line)
            debug_lines.append(re.search(r'DEBUG(.*)', line).group(1))
        elif debug_flag:
            if not re.match(r'^\d{4}-\d{2}-\d{2}', line):
                debug_lines.append(line)
            else:
                json_str = "".join(debug_lines)
                try:
                    json_pool = json.loads(json_str)
                except:
                    print(json_str)
                    json_pool = json_str
                block["ACTION"] = json_pool  
                output.append(block)
                block = {}
                debug_flag = False
                debug_lines = []

    if debug_flag:
        block["ACTION"] = json.loads("".join(debug_lines))
        #output.append(block)

In [103]:
output[6]

[{},
 {},
 {'PANGUGPT THOUGHTS': "I need to conduct a thorough search of online resources to gather accurate and up-to-date information about the Pangu Large Model. I can use the 'google' command to search for information about the model's function and author.",
  'REASONING': "To provide accurate and reliable information about the Pangu Large Model, I need to conduct a thorough search of online resources to gather up-to-date information. The 'google' command is a useful tool for conducting online searches.",
  'PLAN': ["Use the 'google' command to search for information about the Pangu Large Model's function and author.",
   "Analyze the search results to identify relevant information about the model's capabilities and potential applications, as well as the author or authors of the model and their contributions to the field."]}]

In [55]:
json.loads(output[0]['ACTION'])

[{'role': 'system',
  'content': 'You are PanguGPT, an AI assistant that specializes in providing information about the Pangu Large Model, including its function and author.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGOALS:\n\n1. Conduct a thorough search of online resources to gather accurate and up-to-date information about the Pangu Large Model.\n2. Provide a detailed explanation of the function of the Pangu Large Model, including its capabilities and potential applications.\n3. Identify the author or authors of the Pangu Large Model and provide relevant background information about their expertise and contributions to the field.\n4. Ensure that all information provided is reliable, credible, and properly sourced.\n5. Offer additional insights or recommendations based on the gathered information to help the user better understand the Pangu Large Model